In [8]:
# Encryption using QKD for Securing Communication
import random
from qiskit import QuantumCircuit, execute, Aer
# UDF-User Definced Function
def encode_qubits(bits, bases): 
    assert len(bits) == len(bases)    
    encoded_qubits = []
    for bit, base in zip(bits, bases):
        # create new qubit by the encoding scheme
        qCircuit = QuantumCircuit(1, 1)
       
        # check states and bits to decide how to manipulate
        if bit == '0':
            if base == 'Z':
                pass
            elif base == 'X':
                qCircuit.h(0)
        elif bit == '1':
            if base == 'Z':
                qCircuit.x(0)
            elif base == 'X':
                qCircuit.x(0)
                qCircuit.h(0)
        
        encoded_qubits.append(qCircuit)        
    return encoded_qubits

def measure_qubits(qubits, bases):   
    result_bits = []
    
    for qubit, base in zip(qubits, bases):
        # add measurements to qubits with given base
        if base == 'Z':
            qubit.measure(0,0)
        elif base == 'X':
            qubit.h(0)
            qubit.measure(0,0)
            
        # Environments
        jobs = execute(qubit, backend=Aer.get_backend('qasm_simulator'), shots = 1) 
        results = jobs.result()
        counts = results.get_counts()
        measured_bit = max(counts, key=counts.get)        
        result_bits.append(measured_bit)        
    return result_bits

def generate_bits(bitsCounts):
     return [random.choice(['0', '1']) for i in range(bitsCounts)]

def generate_bases(baseCounts):
      return [random.choice(['Z', 'X']) for i in range(baseCounts)]   

def eliminate_differences(bits, indexes):
    result_key = []
    for i in range(len(bits)):
        if i in indexes:
            result_key.append(bits[i])    
    return result_key


In [11]:
#Generating Bits and Bases
countsX=500
alice_bits = generate_bits(countsX)
alice_bases = generate_bases(countsX)

eve_bases = generate_bases(countsX)
eve_bits = measure_qubits(encoded_qubits, eve_bases) 

bob_bases = generate_bases(countsX)
bob_bits = measure_qubits(encoded_qubits, bob_bases)

#Encoding
encoded_qubits = encode_qubits(alice_bits, alice_bases)

print("Alice Bits:",alice_bits)
print("Alice Bases:",alice_bases)

print('Eve Bases:',eve_bases)
print('Eve Bits:', eve_bits)

print('Bob Bases:', bob_bases)
print('Bob Bits:',bob_bits)



Alice Bits: ['0', '1', '0', '0', '0', '0', '0', '0', '1', '1', '0', '1', '1', '1', '0', '0', '1', '0', '0', '0', '0', '1', '0', '1', '0', '0', '1', '1', '1', '0', '0', '1', '0', '0', '0', '1', '1', '0', '0', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '1', '1', '1', '0', '1', '1', '0', '1', '0', '0', '0', '1', '1', '0', '1', '1', '1', '0', '1', '0', '1', '1', '1', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '1', '0', '0', '1', '1', '0', '1', '1', '1', '1', '1', '0', '0', '0', '1', '1', '1', '1', '1', '0', '0', '0', '0', '1', '1', '0', '0', '1', '1', '1', '0', '0', '1', '1', '0', '1', '0', '1', '1', '0', '0', '0', '0', '0', '0', '1', '0', '1', '1', '0', '0', '1', '1', '0', '0', '1', '0', '0', '0', '1', '0', '0', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '1', '1', '1', '1', '0', '0', '0', '0', '1', '1', '0

In [17]:
#Index and Comparisions
same_base_indexes = []
for i in range(len(alice_bases)):
    if alice_bases[i] == bob_bases[i]:
        same_base_indexes.append(i)
        
#Alice Key
alice_key = eliminate_differences(
    alice_bits,
    same_base_indexes
)


# Bob's Key
bob_key = eliminate_differences(
    bob_bits,
    same_base_indexes
)
print('Alice key:', alice_key)
print('Bob key:',bob_key)

#Key comparision
if alice_key == bob_key:
    print("Safe to use:", len(alice_key)) 
else:
    print("Compromised:", len(alice_key)) 

Alice key: ['0', '0', '0', '1', '0', '0', '1', '0', '1', '0', '1', '1', '0', '1', '0', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '0', '1', '0', '1', '0', '0', '1', '1', '1', '1', '1', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '1', '1', '0', '1', '1', '1', '0', '0', '0', '1', '1', '0', '1', '0', '1', '0', '0', '1', '0', '1', '0', '1', '0', '0', '1', '1', '1', '0', '1', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '1', '1', '0', '0', '0', '1', '1', '0', '1', '1', '1', '0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '1', '0', '0', '1', '1', '1', '0', '0', '1', '0', '0', '1', '1', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '1', '1', '0', '1', '1', '1', '1', '1', '0', '1', '1', '0', '0', '0', '0', '0', '1', '0', '1', '1', '0', '1', '0', '1', '1', '0', '0', '1', '0', '0', '1', '1', '1', '0'

In [26]:
#UDF for encrption and decrption
# OPTIONAL - using key to send information (scenario when key is not compromise)
import binascii
def encrypt_message(unencrypted_string, key):
   # bits = bin(int(binascii.hexlify(unencrypted_string.encode('utf-8', 'surrogatepass')), 16))[2:]
    bits = bin(int(binascii.hexlify(unencrypted_string.encode('utf-8', 'surrogatepass').strip()), 16))[2:]
    print(bits)
    bitstring = bits.zfill(8 * ((len(bits) + 7) // 8))
    encrypted_string = ""
    for i in range(len(bitstring)):
        encrypted_string += str( (int(bitstring[i])^ int(key[i])) )
    return encrypted_string
    
def decrypt_message(encrypted_bits, key):
    unencrypted_bits = ""
    for i in range(len(encrypted_bits)):
        unencrypted_bits += str( (int(encrypted_bits[i])^ int(key[i])) )
    i = int(unencrypted_bits, 2)
    hex_string = '%x' % i
    n = len(hex_string)
    bits = binascii.unhexlify(hex_string.zfill(n + (n & 1)))
    unencrypted_string = bits.decode('utf-8', 'surrogatepass')
    return unencrypted_string


In [32]:
# Alice encrypts secret message with key that is already distributed.
secret_message     = 'Quantum Computing is cool :)'
encrypted_message  = (encrypt_message(secret_message, alice_key))

print('Alice message:', secret_message)
print('\nEncrypted message:', encrypted_message)


# Alice sends encrypted message to Bob on public channel ...

1010001011101010110000101101110011101000111010101101101001000000100001101101111011011010111000001110101011101000110100101101110011001110010000001101001011100110010000001100011011011110110111101101100001000000011101000101001
Alice message: Quantum Computing is cool :)

Encrypted message: 01000011110000000001111010000000110100111010110101100100111111000010100100111011100001001011000001100000111110011010101001000111101011101111000001110010011100101010110110010101011001000011011001010110101001001011111011010111


In [34]:
#Decrypting messages, Encoding error fixing
decrypted_message = decrypt_message(encrypted_message, bob_key)
print('Message decrypted by Bob:', decrypted_message)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x83 in position 0: invalid start byte